# Credit Risk Resampling Techniques

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

# Read the CSV and Perform Basic Data Cleaning

In [3]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [4]:
# Load the data
file_path = Path('/Users/jiangao/Desktop/Data Analysis/linear_regression_salary/Module-17-Challenge-Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [5]:
# Create our features
X = df.drop(columns=target)

# Create our target
y = df[target]

In [6]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X = X.copy()
X['home_ownership'] = label_encoder.fit_transform(X['home_ownership'])
X['verification_status'] = label_encoder.fit_transform(X['verification_status'])
X['pymnt_plan'] = label_encoder.fit_transform(X['pymnt_plan'])
X['hardship_flag'] = label_encoder.fit_transform(X['hardship_flag'])
X['debt_settlement_flag'] = label_encoder.fit_transform(X['debt_settlement_flag'])
X['initial_list_status'] = label_encoder.fit_transform(X['initial_list_status'])
X['application_type'] = label_encoder.fit_transform(X['application_type'])
X = X.drop(columns = "next_pymnt_d")
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,pymnt_plan,dti,delinq_2yrs,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,3,66000.0,1,Mar-2019,0,27.24,0.0,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0
1,25000.0,0.2000,929.09,1,105000.0,2,Mar-2019,0,20.23,0.0,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0
2,20000.0,0.2000,529.88,1,56000.0,2,Mar-2019,0,24.26,0.0,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0
3,10000.0,0.1640,353.55,3,92000.0,2,Mar-2019,0,31.44,0.0,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0
4,22000.0,0.1474,520.39,1,52000.0,0,Mar-2019,0,18.76,0.0,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0


In [7]:
# Months dictionary
months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5,
    "Jun-2019": 6,
    "Jul-2019": 7,
    "Aug-2019": 8,
    "Sep-2019": 9,
    "Oct-2019": 10,
    "Nov-2019": 11,
    "Dec-2019": 12,
}

In [8]:
# Months' names encoded using the dictionary values
X["months_num"] = X["issue_d"].apply(lambda x: months_num[x])
X = X.drop(columns='issue_d')
X.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,months_num
0,10500.0,0.1719,375.35,3,66000.0,1,0,27.24,0.0,0.0,...,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0,3
1,25000.0,0.2000,929.09,1,105000.0,2,0,20.23,0.0,0.0,...,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0,3
2,20000.0,0.2000,529.88,1,56000.0,2,0,24.26,0.0,0.0,...,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0,3
3,10000.0,0.1640,353.55,3,92000.0,2,0,31.44,0.0,1.0,...,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0,3
4,22000.0,0.1474,520.39,1,52000.0,0,0,18.76,0.0,1.0,...,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0,3


In [9]:
X_combo = X

In [10]:
from sklearn.preprocessing import StandardScaler
data_scaler = StandardScaler()
X = data_scaler.fit_transform(X)
X = pd.DataFrame(X)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
0,-0.601093,0.917984,-0.365558,1.261248,-0.192195,0.458915,0.0,0.270401,-0.303143,-0.656492,...,2.062779,-0.374103,0.0,-0.748655,-0.403213,-1.035041,0.122936,0.0,0.0,1.712464
1,0.809787,1.501829,1.556747,-0.863458,0.145236,1.849543,0.0,-0.076645,-0.303143,-0.656492,...,0.576065,2.595637,0.0,0.318420,-0.012153,0.427908,-0.128054,0.0,0.0,1.712464
2,0.323277,1.501829,0.170892,-0.863458,-0.278716,1.849543,0.0,0.122869,-0.303143,-0.656492,...,0.576065,-0.374103,0.0,-0.774811,-0.272784,-0.829780,-0.246838,0.0,0.0,1.712464
3,-0.649744,0.753843,-0.441237,1.261248,0.032759,1.849543,0.0,0.478331,-0.303143,0.662568,...,0.576065,2.595637,0.0,-0.573252,0.129742,-0.374475,0.407857,0.0,0.0,1.712464
4,0.517881,0.408938,0.137947,-0.863458,-0.313324,-0.931713,0.0,-0.149420,-0.303143,0.662568,...,-0.910648,-0.374103,0.0,0.050396,-0.617197,-0.079646,-0.701016,0.0,0.0,1.712464


In [11]:
X.describe()

,0,1,2,3,4,5,6,7,8,9,...,74,75,76,77,78,79,80,81,82,83
count,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,68817.0,6.881700e+04,6.881700e+04,6.881700e+04,...,6.881700e+04,6.881700e+04,68817.0,6.881700e+04,6.881700e+04,6.881700e+04,6.881700e+04,68817.0,68817.0,6.881700e+04
mean,-7.361679e-18,-1.199557e-15,6.590079e-17,-7.332018e-16,6.016378e-16,-6.996491e-15,0.0,-9.826434e-17,-2.177913e-15,-1.431710e-15,...,4.623001e-16,3.377942e-15,0.0,-1.104218e-16,-2.881793e-17,2.136167e-17,-1.063458e-16,0.0,0.0,-3.517380e-13
std,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,0.0,1.000007e+00,1.000007e+00,1.000007e+00,...,1.000007e+00,1.000007e+00,0.0,1.000007e+00,1.000007e+00,1.000007e+00,1.000007e+00,0.0,0.0,1.000007e+00
min,-1.525462e+00,-1.407007e+00,-1.561349e+00,-1.925812e+00,-7.628861e-01,-9.317127e-01,0.0,-1.078175e+00,-3.031425e-01,-6.564915e-01,...,-9.106485e-01,-3.741034e-01,0.0,-1.070671e+00,-1.064750e+00,-1.105949e+00,-1.091003e+00,0.0,0.0,-9.762264e-01
25%,-7.470458e-01,-8.231618e-01,-7.461036e-01,-8.634582e-01,-3.306281e-01,-9.317127e-01,0.0,-3.905201e-01,-3.031425e-01,-6.564915e-01,...,-9.106485e-01,-3.741034e-01,0.0,-7.419643e-01,-6.070206e-01,-6.767678e-01,-6.444985e-01,0.0,0.0,-9.762264e-01
50%,-1.632334e-01,-2.019175e-01,-2.641560e-01,-8.634582e-01,-1.316302e-01,4.589150e-01,0.0,-9.991302e-02,-3.031425e-01,-6.564915e-01,...,-3.159629e-01,-3.741034e-01,0.0,-3.284273e-01,-2.784825e-01,-2.849066e-01,-2.692881e-01,0.0,0.0,3.681190e-01
75%,7.124852e-01,5.813905e-01,5.812919e-01,1.261248e+00,1.365843e-01,4.589150e-01,0.0,2.416864e-01,-3.031425e-01,6.625677e-01,...,5.760654e-01,-3.741034e-01,0.0,4.854937e-01,2.654159e-01,3.569994e-01,3.292235e-01,0.0,0.0,3.681190e-01
max,2.269318e+00,3.754100e+00,4.150440e+00,1.261248e+00,7.535346e+01,1.849543e+00,0.0,4.837948e+01,2.475388e+01,5.938804e+00,...,2.062779e+00,1.150486e+01,0.0,1.598875e+01,2.150495e+01,1.790119e+01,2.690921e+01,0.0,0.0,1.712464e+00


In [12]:
y = df["loan_status"]
y.head()

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [13]:
# Check the balance of our target values
y[:].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [14]:
# Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({'low_risk': 51366, 'high_risk': 246})

# Oversampling

In this section, you will compare two oversampling algorithms to determine which algorithm results in the best performance. You will oversample the data using the naive random oversampling algorithm and the SMOTE algorithm. For each algorithm, be sure to complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

### Naive Random Oversampling

In [15]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [16]:
y_resampled

0          low_risk
1          low_risk
2          low_risk
3          low_risk
4          low_risk
            ...    
102727    high_risk
102728    high_risk
102729    high_risk
102730    high_risk
102731    high_risk
Name: loan_status, Length: 102732, dtype: object

In [17]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [18]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   76,    25],
       [ 3507, 13597]])

In [19]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.7737177453713566

In [20]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.75      0.79      0.04      0.77      0.60       101
   low_risk       1.00      0.79      0.75      0.89      0.77      0.60     17104

avg / total       0.99      0.79      0.75      0.88      0.77      0.60     17205



### SMOTE Oversampling

In [21]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1).fit_resample(
    X_train, y_train
)
Counter(y_resampled)

Counter({'low_risk': 51366, 'high_risk': 51366})

In [22]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [23]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7666815822134131

In [24]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   74,    27],
       [ 3409, 13695]])

In [25]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.73      0.80      0.04      0.77      0.58       101
   low_risk       1.00      0.80      0.73      0.89      0.77      0.59     17104

avg / total       0.99      0.80      0.73      0.88      0.77      0.59     17205



# Undersampling

In this section, you will test an undersampling algorithms to determine which algorithm results in the best performance compared to the oversampling algorithms above. You will undersample the data using the Cluster Centroids algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [26]:
# Resample the data using the ClusterCentroids resampler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [27]:
# Train the Logistic Regression model using the resampled data
from imblearn.under_sampling import RandomUnderSampler
ros = RandomUnderSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'high_risk': 246, 'low_risk': 246})

In [28]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.7666815822134131

In [29]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[   74,    27],
       [ 3409, 13695]])

In [30]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.02      0.73      0.80      0.04      0.77      0.58       101
   low_risk       1.00      0.80      0.73      0.89      0.77      0.59     17104

avg / total       0.99      0.80      0.73      0.88      0.77      0.59     17205



# Combination (Over and Under) Sampling

In this section, you will test a combination over- and under-sampling algorithm to determine if the algorithm results in the best performance compared to the other sampling algorithms above. You will resample the data using the SMOTEENN algorithm and complete the folliowing steps:

1. View the count of the target classes using `Counter` from the collections library. 
3. Use the resampled data to train a logistic regression model.
3. Calculate the balanced accuracy score from sklearn.metrics.
4. Print the confusion matrix from sklearn.metrics.
5. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.

Note: Use a random state of 1 for each sampling algorithm to ensure consistency between tests

In [31]:
# Resample the training data with SMOTEENN
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_combo, y, random_state=1)

from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_combo, y)
Counter(y_resampled)

Counter({'high_risk': 68458, 'low_risk': 62022})

In [32]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [33]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)

from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_test, y_pred)

0.6467579814576406

In [36]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  73,   28],
       [7342, 9762]])

In [37]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.72      0.57      0.02      0.64      0.42       101
   low_risk       1.00      0.57      0.72      0.73      0.64      0.41     17104

avg / total       0.99      0.57      0.72      0.72      0.64      0.41     17205

